In [1]:
%pip install torch torchvision datasets transformers


Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

# Load the MNIST dataset
dataset = load_dataset('ylecun/mnist')

# Dataset and DataLoader setup
train_data = dataset['train']
test_data = dataset['test']

# Normalize and transform the data into tensor form
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# Custom dataset class to apply the transformations
class MNISTDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]['image']
        label = self.data[idx]['label']
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Create DataLoader for training and test datasets
train_dataset = MNISTDataset(train_data, transform)
test_dataset = MNISTDataset(test_data, transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

README.md:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [2]:
import torch.nn as nn
import torch.optim as optim

# Define the MLP model class
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # Input layer (784 input features)
        self.fc2 = nn.Linear(128, 64)       # Hidden layer
        self.fc3 = nn.Linear(64, 10)        # Output layer (10 classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image into a 1D vector
        x = self.relu(self.fc1(x))  # First hidden layer
        x = self.relu(self.fc2(x))  # Second hidden layer
        x = self.fc3(x)  # Output layer (logits)
        return self.softmax(x)  # Softmax to get probabilities

# Initialize the model
model = MLP()

In [3]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, optimizer, criterion, epochs=5):
    model.train()  # Set model to training mode
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()  # Zero the gradients

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

# Train the model
train_model(model, train_loader, optimizer, criterion)

Epoch 1, Loss: 1.6265175746702183
Epoch 2, Loss: 1.547849739538327
Epoch 3, Loss: 1.529253351154612
Epoch 4, Loss: 1.518996642596686
Epoch 5, Loss: 1.5123095133665527


In [4]:
# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No need to compute gradients during evaluation
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the class with highest probability
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total * 100
    print(f'Test Accuracy: {accuracy:.2f}%')

# Evaluate the model on test set
evaluate_model(model, test_loader)

Test Accuracy: 95.39%


In [5]:
# Save MLP model
torch.save(model.state_dict(), "mlp_model.pt")
print("MLP model saved successfully.")

MLP model saved successfully.
